In [315]:
import pandas as pd
import random
import numpy as np
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score, precision_score, classification_report, precision_recall_curve, confusion_matrix

1. взять любой набор данных для бинарной классификации (можно скачать один из модельных с https://archive.ics.uci.edu/ml/datasets.php)

In [316]:
df = pd.read_csv('secondary_data.csv', ';')

C:\Users\ASUS\AppData\Local\Temp\ipykernel_8872\1633459544.py:1: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  df = pd.read_csv('secondary_data.csv', ';')


In [317]:
df.head()

,class,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,...,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
0,p,15.26,x,g,o,f,e,NaN,w,16.95,...,s,y,w,u,w,t,g,NaN,d,w
1,p,16.60,x,g,o,f,e,NaN,w,17.99,...,s,y,w,u,w,t,g,NaN,d,u
2,p,14.07,x,g,o,f,e,NaN,w,17.80,...,s,y,w,u,w,t,g,NaN,d,w
3,p,14.17,f,h,e,f,e,NaN,w,15.77,...,s,y,w,u,w,t,p,NaN,d,w
4,p,14.64,x,h,o,f,e,NaN,w,16.53,...,s,y,w,u,w,t,p,NaN,d,w


In [318]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61069 entries, 0 to 61068
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   class                 61069 non-null  object 
 1   cap-diameter          61069 non-null  float64
 2   cap-shape             61069 non-null  object 
 3   cap-surface           46949 non-null  object 
 4   cap-color             61069 non-null  object 
 5   does-bruise-or-bleed  61069 non-null  object 
 6   gill-attachment       51185 non-null  object 
 7   gill-spacing          36006 non-null  object 
 8   gill-color            61069 non-null  object 
 9   stem-height           61069 non-null  float64
 10  stem-width            61069 non-null  float64
 11  stem-root             9531 non-null   object 
 12  stem-surface          22945 non-null  object 
 13  stem-color            61069 non-null  object 
 14  veil-type             3177 non-null   object 
 15  veil-color         

In [319]:
df['class'].value_counts()

p    33888
e    27181
Name: class, dtype: int64

2. сделать feature engineering

<span style="color:#8B0000"> У некоторых признаков заполняем пропуски рандомными значениями </span>

In [320]:
cat_nans = ['cap-surface', 'gill-attachment', 'gill-spacing','ring-type']

In [321]:
for cat in cat_nans:
    
    df[cat] = df[cat].fillna(0)
    cat_types=list(df[cat].unique())
    cat_types.remove(0)
    df[cat] = df[cat].apply(lambda x: x if x!=0 else random.choice(cat_types))


<span style="color:#8B0000"> Признаки, в которых ОЧЕНЬ много пропусков, в основном удалим (spore-print-color,veil-type,stem-surface,stem-root) кроме:<br>
Veil-color (цвет "вуали") сделаем равным цвету ножки гриба<br>
</span>

In [322]:
feat_to_del = ['spore-print-color','veil-type','stem-surface','stem-root']
for f in feat_to_del:
    df.drop(f, inplace=True, axis=1)

In [323]:
df.loc[df['veil-color'].isna(), 'veil-color'] = df['stem-color']

In [324]:
df['veil-color'].unique()

array(['w', 'y', 'n', 'u', 'b', 'l', 'r', 'p', 'e', 'k', 'g', 'o', 'f'],
      dtype=object)

In [325]:
df.isna().sum().sum()

0

In [326]:
df.loc[df['has-ring']=='t', 'has-ring'] = 1
df.loc[df['has-ring']=='f', 'has-ring'] = 0
df['has-ring'] = df['has-ring'].astype('int32')

In [327]:
df['has-ring'].value_counts()

0    45890
1    15179
Name: has-ring, dtype: int64

In [328]:
df.loc[df['does-bruise-or-bleed']=='t', 'does-bruise-or-bleed'] = 1
df.loc[df['does-bruise-or-bleed']=='f', 'does-bruise-or-bleed'] = 0
df['does-bruise-or-bleed'] = df['does-bruise-or-bleed'].astype('int32')

In [329]:
df.loc[df['class']=='p', 'class'] = 1
df.loc[df['class']=='e', 'class'] = 0
df['class'] = df['class'].astype('int32')

In [330]:
df['class'].value_counts()

1    33888
0    27181
Name: class, dtype: int64

3. обучить любой классификатор (какой вам нравится)

<span style="color:#8B0000"> Поскольку практически все признаки тут категориальные, будем обучать CatBoost</span>

In [331]:
#разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(df.drop(['class'], axis=1), df['class'], random_state=26, shuffle=True)

<span style="color:#8B0000"> *отбираем категор. и веществ. признаки*</span>

In [332]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61069 entries, 0 to 61068
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   class                 61069 non-null  int32  
 1   cap-diameter          61069 non-null  float64
 2   cap-shape             61069 non-null  object 
 3   cap-surface           61069 non-null  object 
 4   cap-color             61069 non-null  object 
 5   does-bruise-or-bleed  61069 non-null  int32  
 6   gill-attachment       61069 non-null  object 
 7   gill-spacing          61069 non-null  object 
 8   gill-color            61069 non-null  object 
 9   stem-height           61069 non-null  float64
 10  stem-width            61069 non-null  float64
 11  stem-color            61069 non-null  object 
 12  veil-color            61069 non-null  object 
 13  has-ring              61069 non-null  int32  
 14  ring-type             61069 non-null  object 
 15  habitat            

In [333]:
categorical_columns = ['cap-shape', 'cap-surface', 'cap-color', 'gill-attachment', \
                      'gill-spacing', 'gill-color', 'stem-color', 'veil-color', 'ring-type', 'habitat', 'season']
continuous_columns = ['cap-diameter', 'does-bruise-or-bleed', 'stem-height', 'stem-width', 'has-ring']

<span style="color:#8B0000">*обучаем модель CatBoost*</span>

In [334]:
modelCB = CatBoostClassifier(iterations=20, thread_count=2, random_state=26, silent=True)

In [335]:
modelCB = modelCB.fit(X_train, y_train, cat_features=categorical_columns)

In [336]:
preds = modelCB.predict_proba(X_test)[:, 1]

In [337]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)

fscore = (2 * precision * recall) / (precision + recall)
ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')

Best Threshold=0.5015015549614082, F-Score=0.952, Precision=0.945, Recall=0.960


In [338]:
results = pd.DataFrame({'Step': ['Usual Classification with labeled data'],
                        'Fscore': [fscore[ix]],
              'precision': [precision[ix]],
              'recall': [recall[ix]],
              'threshold': [thresholds[ix]]})

<span style="color:#8B0000"> *Результаты получились очень даже неплохие*</span>

4. Далее разделить ваш набор данных на два множества: P (positives) и U (unlabeled). Причем брать нужно не все положительные (класс 1) примеры, а только лишь часть

In [339]:
mod_df = df.copy()
pos_ind = np.where(mod_df.iloc[:,0].values == 1)[0] #отбираем индексы только positives
np.random.shuffle(pos_ind) #перемешиваем их
pos_sample_len = int(0.25*len(pos_ind)) #устанавливаем кол-во новых positives (только 25% от элементов класса 1)
print(f'Use {pos_sample_len}/{len(pos_ind)} as positives')
pos_sample = pos_ind[:pos_sample_len] #отбираем 25% от элементов класса 1

Use 8472/33888 as positives


In [340]:
mod_df['class_test'] = -1 #ВСЕ элементы помечаем как unlabeled
mod_df.loc[pos_sample, 'class_test'] = 1 #по отобранным ранее индексам 8472 эл-та помечаем как positive
print(f'Target:\n', mod_df.iloc[:,-1].value_counts())

Target:
 -1    52597
 1     8472
Name: class_test, dtype: int64


In [341]:
mod_df.head(5)

,class,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,stem-width,stem-color,veil-color,has-ring,ring-type,habitat,season,class_test
0,1,15.26,x,g,o,0,e,d,w,16.95,17.09,w,w,1,g,d,w,1
1,1,16.60,x,g,o,0,e,d,w,17.99,18.19,w,w,1,g,d,u,-1
2,1,14.07,x,g,o,0,e,c,w,17.80,17.74,w,w,1,g,d,w,1
3,1,14.17,f,h,e,0,e,d,w,15.77,15.98,w,w,1,p,d,w,-1
4,1,14.64,x,h,o,0,e,d,w,16.53,17.20,w,w,1,p,d,w,-1


In [342]:
x_data = mod_df.iloc[:,1:-1].values #dataframe без разметки (исходной и новой)
y_labeled = mod_df.iloc[:,-1].values #новая разметка по классам 1/-1
y_positive = mod_df.iloc[:,0].values #исходная разметка по классам

In [343]:
mod_df = mod_df.sample(frac=1) #перемешиваем
neg_sample = mod_df[mod_df['class_test']==-1][:len(mod_df[mod_df['class_test']==1])] #sample с unlabeled эл-тами 8472шт
sample_test = mod_df[mod_df['class_test']==-1][len(mod_df[mod_df['class_test']==1]):] #sample с остальными unlabeled эл-тами
pos_sample = mod_df[mod_df['class_test']==1] #sample с positive эл-тами 8472шт
sample_train = pd.concat([neg_sample, pos_sample]).sample(frac=1)

In [344]:
sample_train.head(5)

,class,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,stem-width,stem-color,veil-color,has-ring,ring-type,habitat,season,class_test
40673,1,3.96,x,t,n,0,s,c,n,5.80,9.52,w,n,1,z,d,a,-1
45653,0,1.08,p,g,g,0,s,f,g,3.06,1.58,w,w,0,f,d,w,-1
29524,1,2.64,f,w,n,1,d,c,n,2.25,4.61,n,n,0,f,d,a,-1
28397,1,13.65,x,t,n,1,d,c,n,4.83,24.38,r,r,0,f,g,a,-1
16455,0,5.27,f,y,n,0,s,d,w,5.07,8.23,w,w,0,f,d,a,-1


In [345]:
sample_train.loc[sample_train['class_test']==-1, 'class_test'] = 0

In [346]:
sample_test.head(5)

,class,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,stem-width,stem-color,veil-color,has-ring,ring-type,habitat,season,class_test
9323,0,3.37,b,g,n,0,x,d,w,5.08,3.24,y,y,0,f,g,a,-1
15612,1,6.47,x,s,r,1,s,d,w,5.98,16.74,n,n,0,f,d,a,-1
4895,0,1.67,x,l,b,0,e,d,u,5.07,4.21,w,w,0,f,d,u,-1
2857,0,15.51,p,y,w,0,s,d,w,25.65,19.34,n,n,1,m,m,u,-1
34571,1,1.75,f,h,n,0,s,f,n,8.26,2.83,y,y,0,f,g,a,-1


5. применить random negative sampling для построения классификатора в новых условиях

In [347]:
modelCB = modelCB.fit(sample_train.iloc[:,1:-1], sample_train.iloc[:, -1], cat_features=categorical_columns)
y_predict = modelCB.predict(sample_test.iloc[:,1:-1])

In [348]:
y_predict

array([0, 1, 0, ..., 0, 0, 0], dtype=int64)

In [349]:
precision, recall, thresholds = precision_recall_curve(sample_test.iloc[:,0], y_predict)

fscore = (2 * precision * recall) / (precision + recall)
ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')

Best Threshold=1, F-Score=0.866, Precision=0.931, Recall=0.809


In [350]:
results = results.append({'Step': 'RNS 25%',
                          'Fscore': fscore[ix],
              'precision': precision[ix],
              'recall': recall[ix],
              'threshold': thresholds[ix]}, ignore_index=True)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_8872\3985584856.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({'Step': 'RNS 25%',


### <span style="color:#8B0000"> Ниже решение аналогичное решению, предложенному в прикрепленном к ДЗ файле. Я с ним не согласна, т.к. модели там передаются для обучения *исходные* классы. В чём тогда смысл того, что мы создавали "фиктивные" unlabeled классы?</span>

In [351]:
modelCB = modelCB.fit(sample_train.iloc[:,1:-1], sample_train.iloc[:, 0], cat_features=categorical_columns)
y_predict = modelCB.predict(sample_test.iloc[:,1:-1])

In [352]:
y_predict

array([0, 1, 0, ..., 0, 0, 0], dtype=int64)

In [353]:
precision, recall, thresholds = precision_recall_curve(sample_test.iloc[:,0], y_predict)

fscore = (2 * precision * recall) / (precision + recall)
ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')

Best Threshold=1, F-Score=0.888, Precision=0.814, Recall=0.977


In [354]:
results = results.append({'Step': 'RNS 25%(like in example)',
                          'Fscore': fscore[ix],
              'precision': precision[ix],
              'recall': recall[ix],
              'threshold': thresholds[ix]}, ignore_index=True)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_8872\3371570068.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({'Step': 'RNS 25%(like in example)',


6. сравнить качество с решением из пункта 4 (построить отчет - таблицу метрик)

In [355]:
results.T

,0,1,2
Step,Usual Classification with labeled data,RNS 25%,RNS 25%(like in example)
Fscore,0.952281,0.865639,0.888017
precision,0.944657,0.930761,0.81388
recall,0.960028,0.809035,0.977015
threshold,0.501502,1.0,1.0


7. поэкспериментировать с долей P на шаге 6 (как будет меняться качество модели при уменьшении/увеличении размера P)

### 35%

In [356]:
mod_df = df.copy()
pos_ind = np.where(mod_df.iloc[:,0].values == 1)[0] #отбираем индексы только positives
np.random.shuffle(pos_ind) #перемешиваем их
pos_sample_len = int(0.35*len(pos_ind)) #устанавливаем кол-во новых positives (только 35% от элементов класса 1)
print(f'Use {pos_sample_len}/{len(pos_ind)} as positives')
pos_sample = pos_ind[:pos_sample_len] #отбираем 25% от элементов класса 1

Use 11860/33888 as positives


In [357]:
mod_df['class_test'] = -1 #ВСЕ элементы помечаем как unlabeled
mod_df.loc[pos_sample, 'class_test'] = 1 #по отобранным ранее индексам 11860 эл-та помечаем как positive
print(f'Target:\n', mod_df.iloc[:,-1].value_counts())

Target:
 -1    49209
 1    11860
Name: class_test, dtype: int64


In [358]:
mod_df = mod_df.sample(frac=1) #перемешиваем
neg_sample = mod_df[mod_df['class_test']==-1][:len(mod_df[mod_df['class_test']==1])] #sample с unlabeled эл-тами 11860шт
sample_test = mod_df[mod_df['class_test']==-1][len(mod_df[mod_df['class_test']==1]):] #sample с остальными unlabeled эл-тами
pos_sample = mod_df[mod_df['class_test']==1] #sample с positive эл-тами 11860шт
sample_train = pd.concat([neg_sample, pos_sample]).sample(frac=1)

In [359]:
sample_train.loc[sample_train['class_test']==-1, 'class_test'] = 0

In [360]:
modelCB = modelCB.fit(sample_train.iloc[:,1:-1], sample_train.iloc[:, -1], cat_features=categorical_columns)
y_predict = modelCB.predict(sample_test.iloc[:,1:-1])

In [361]:
precision, recall, thresholds = precision_recall_curve(sample_test.iloc[:,0], y_predict)

fscore = (2 * precision * recall) / (precision + recall)
ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')

Best Threshold=1, F-Score=0.883, Precision=0.932, Recall=0.840


In [362]:
results = results.append({'Step': 'RNS 35%',
                          'Fscore': fscore[ix],
              'precision': precision[ix],
              'recall': recall[ix],
              'threshold': thresholds[ix]}, ignore_index=True)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_8872\2822183774.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({'Step': 'RNS 35%',


### 50%

In [363]:
mod_df = df.copy()
pos_ind = np.where(mod_df.iloc[:,0].values == 1)[0] #отбираем индексы только positives
np.random.shuffle(pos_ind) #перемешиваем их
pos_sample_len = int(0.5*len(pos_ind)) #устанавливаем кол-во новых positives (только 50% от элементов класса 1)
print(f'Use {pos_sample_len}/{len(pos_ind)} as positives')
pos_sample = pos_ind[:pos_sample_len] #отбираем 50% от элементов класса 1

mod_df['class_test'] = -1 #ВСЕ элементы помечаем как unlabeled
mod_df.loc[pos_sample, 'class_test'] = 1 #по отобранным ранее индексам 16944 эл-та помечаем как positive
print(f'Target:\n', mod_df.iloc[:,-1].value_counts())

mod_df = mod_df.sample(frac=1) #перемешиваем
neg_sample = mod_df[mod_df['class_test']==-1][:len(mod_df[mod_df['class_test']==1])] #sample с unlabeled эл-тами 16944шт
sample_test = mod_df[mod_df['class_test']==-1][len(mod_df[mod_df['class_test']==1]):] #sample с остальными unlabeled эл-тами
pos_sample = mod_df[mod_df['class_test']==1] #sample с positive эл-тами 16944шт
sample_train = pd.concat([neg_sample, pos_sample]).sample(frac=1)
sample_train.loc[sample_train['class_test']==-1, 'class_test'] = 0

modelCB = modelCB.fit(sample_train.iloc[:,1:-1], sample_train.iloc[:, -1], cat_features=categorical_columns)
y_predict = modelCB.predict(sample_test.iloc[:,1:-1])

precision, recall, thresholds = precision_recall_curve(sample_test.iloc[:,0], y_predict)

fscore = (2 * precision * recall) / (precision + recall)
ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')

results = results.append({'Step': 'RNS 50%',
                          'Fscore': fscore[ix],
              'precision': precision[ix],
              'recall': recall[ix],
              'threshold': thresholds[ix]}, ignore_index=True)

Use 16944/33888 as positives
Target:
 -1    44125
 1    16944
Name: class_test, dtype: int64
Best Threshold=1, F-Score=0.914, Precision=0.948, Recall=0.882


C:\Users\ASUS\AppData\Local\Temp\ipykernel_8872\1278626661.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({'Step': 'RNS 50%',


### 75%

In [364]:
mod_df = df.copy()
pos_ind = np.where(mod_df.iloc[:,0].values == 1)[0] #отбираем индексы только positives
np.random.shuffle(pos_ind) #перемешиваем их
pos_sample_len = int(0.75*len(pos_ind)) #устанавливаем кол-во новых positives (только 75% от элементов класса 1)
print(f'Use {pos_sample_len}/{len(pos_ind)} as positives')
pos_sample = pos_ind[:pos_sample_len] #отбираем 75% от элементов класса 1

mod_df['class_test'] = -1 #ВСЕ элементы помечаем как unlabeled
mod_df.loc[pos_sample, 'class_test'] = 1 #по отобранным ранее индексам 25416 эл-та помечаем как positive
print(f'Target:\n', mod_df.iloc[:,-1].value_counts())

mod_df = mod_df.sample(frac=1) #перемешиваем
neg_sample = mod_df[mod_df['class_test']==-1][:len(mod_df[mod_df['class_test']==1])] #sample с unlabeled эл-тами 25416шт
sample_test = mod_df[mod_df['class_test']==-1][len(mod_df[mod_df['class_test']==1]):] #sample с остальными unlabeled эл-тами
pos_sample = mod_df[mod_df['class_test']==1] #sample с positive эл-тами 25416шт
sample_train = pd.concat([neg_sample, pos_sample]).sample(frac=1)
sample_train.loc[sample_train['class_test']==-1, 'class_test'] = 0

modelCB = modelCB.fit(sample_train.iloc[:,1:-1], sample_train.iloc[:, -1], cat_features=categorical_columns)
y_predict = modelCB.predict(sample_test.iloc[:,1:-1])

precision, recall, thresholds = precision_recall_curve(sample_test.iloc[:,0], y_predict)

fscore = (2 * precision * recall) / (precision + recall)
ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')

results = results.append({'Step': 'RNS 75%',
                          'Fscore': fscore[ix],
              'precision': precision[ix],
              'recall': recall[ix],
              'threshold': thresholds[ix]}, ignore_index=True)

Use 25416/33888 as positives
Target:
 -1    35653
 1    25416
Name: class_test, dtype: int64
Best Threshold=1, F-Score=0.895, Precision=0.878, Recall=0.914


C:\Users\ASUS\AppData\Local\Temp\ipykernel_8872\3548564607.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({'Step': 'RNS 75%',


### 10%

In [365]:
mod_df = df.copy()
pos_ind = np.where(mod_df.iloc[:,0].values == 1)[0] #отбираем индексы только positives
np.random.shuffle(pos_ind) #перемешиваем их
pos_sample_len = int(0.1*len(pos_ind)) #устанавливаем кол-во новых positives (только 10% от элементов класса 1)
print(f'Use {pos_sample_len}/{len(pos_ind)} as positives')
pos_sample = pos_ind[:pos_sample_len] #отбираем 10% от элементов класса 1

mod_df['class_test'] = -1 #ВСЕ элементы помечаем как unlabeled
mod_df.loc[pos_sample, 'class_test'] = 1 #по отобранным ранее индексам 3388 эл-та помечаем как positive
print(f'Target:\n', mod_df.iloc[:,-1].value_counts())

mod_df = mod_df.sample(frac=1) #перемешиваем
neg_sample = mod_df[mod_df['class_test']==-1][:len(mod_df[mod_df['class_test']==1])] #sample с unlabeled эл-тами 3388шт
sample_test = mod_df[mod_df['class_test']==-1][len(mod_df[mod_df['class_test']==1]):] #sample с остальными unlabeled эл-тами
pos_sample = mod_df[mod_df['class_test']==1] #sample с positive эл-тами 3388шт
sample_train = pd.concat([neg_sample, pos_sample]).sample(frac=1)
sample_train.loc[sample_train['class_test']==-1, 'class_test'] = 0

modelCB = modelCB.fit(sample_train.iloc[:,1:-1], sample_train.iloc[:, -1], cat_features=categorical_columns)
y_predict = modelCB.predict(sample_test.iloc[:,1:-1])

precision, recall, thresholds = precision_recall_curve(sample_test.iloc[:,0], y_predict)

fscore = (2 * precision * recall) / (precision + recall)
ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')

results = results.append({'Step': 'RNS 10%',
                          'Fscore': fscore[ix],
              'precision': precision[ix],
              'recall': recall[ix],
              'threshold': thresholds[ix]}, ignore_index=True)

Use 3388/33888 as positives
Target:
 -1    57681
 1     3388
Name: class_test, dtype: int64
Best Threshold=1, F-Score=0.835, Precision=0.923, Recall=0.762


C:\Users\ASUS\AppData\Local\Temp\ipykernel_8872\2112670908.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({'Step': 'RNS 10%',


In [366]:
results = results.T
results = results[[0,2,6,1,3,4,5]]

In [367]:
results

,0,2,6,1,3,4,5
Step,Usual Classification with labeled data,RNS 25%(like in example),RNS 10%,RNS 25%,RNS 35%,RNS 50%,RNS 75%
Fscore,0.952281,0.888017,0.834514,0.865639,0.883395,0.913529,0.895314
precision,0.944657,0.81388,0.922807,0.930761,0.931909,0.947714,0.877639
recall,0.960028,0.977015,0.761641,0.809035,0.839682,0.881725,0.913716
threshold,0.501502,1.0,1.0,1.0,1.0,1.0,1.0


<span style="color:#8B0000">Отсечка везде получилась 1 - модель абсолютна уверена в своих решениях с вероятностью 1. Чуть вероятность меньше 1 - это уже скорее класс 0.<br><br>
Наилучшие показатели (причем все) наблюдаются тогда, когда от позитивов берётся только 35%.<br>
Более слабые результаты на отметке 10% легко объяснить. У модели не так много данных для обучения + больше вероятность, что в Unlabeled класс попадут Positives (т.к. мы от них взяли только 10%, 90% осталось)<br>
Падение метрик при отметке 75% затрудняюсь объяснить: по идее у модели теперь много данных + в Unlabeled будет мало positives, должны быть хорошие метрики. Вероятно, дело в том, что на Test Sample осталось слишком мало данных.<br>
Следовательно, золотая середина - 35%.<br><br></span>

<span style="color:#8B0000">В целом результаты получились довольно неплохие, даже если брать 10%. Мне приходят в голову 3 причины:<br>
* Модель CatBoost хороша
* Классы 0 и 1 достаточно отличаются друг от друга (удачный датасет)
* Данных много </span>